In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.utils import resample


In [29]:
# Pre set values for max cols and chart size

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

plt.rcParams["figure.figsize"] = (15,5)

In [30]:
# Read training data

data=pd.read_csv('train_values.csv')
data.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [31]:
# Read table with target variable 

label=pd.read_csv('train_labels.csv')
label.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [32]:
# Add Target variable to training data 

data['damage']=label['damage_grade']
data.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [33]:
# data=data.sample(5000,replace=True)

In [34]:
# Check distribution of target variable to find out if there is class imbaance problem
data['damage'].value_counts()

2    148259
3     87218
1     25124
Name: damage, dtype: int64

In [35]:
# Define X and y variables
X=pd.get_dummies(data.loc[:,:'has_secondary_use_other'])
y=data['damage'].astype(int)

In [36]:
# Split the data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=256)

#### Upsample the minority training data

In [53]:
# Separate majority and minority classes
filt1=data['damage']==1
filt2=data['damage']==2
filt3=data['damage']==3

df_majority2 = data[filt2]
df_majority3 = data[filt3]
df_minority  = data[filt1]
 
# Downsample majority class, damage=2 
df_majority_downsampled2 = resample(df_majority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=25124,    # to match majority class
                                 random_state=123) # reproducible results
 
# Downsample majority class, damage=3 
df_majority_downsampled3 = resample(df_majority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=25124,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority classes
df_upsampled = pd.concat([df_minority, df_majority_downsampled2,df_majority_downsampled3])
 
# Display new class counts
df_upsampled.damage.value_counts()


3    25124
2    25124
1    25124
Name: damage, dtype: int64

In [54]:
# Define X and y variables
X=pd.get_dummies(df_upsampled.loc[:,:'has_secondary_use_other'])
y=df_upsampled['damage'].astype(int)

In [55]:
# Split the data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=256)

In [56]:
# Random Forest Classifier
clf=RandomForestClassifier()
clf.fit(X_train,y_train)

prediction=clf.predict(X_test)
print(np.unique(prediction))

C:\Users\Preeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[1 2 3]


In [57]:
pred=pd.DataFrame(prediction)
pred[0].value_counts()

1    5263
3    4937
2    4875
Name: 0, dtype: int64

In [58]:
# Format the prediction as per submission requirement
pred['building_id']=data['building_id']
pred.rename(columns={0:'damage_grade'},inplace=True)
pred=pred[['building_id','damage_grade']]
pred.head()

,building_id,damage_grade
0,802906,1
1,28830,2
2,94947,3
3,590882,2
4,201944,1


In [59]:
cross_validate(clf,X_train,y_train,cv=5)

{'fit_time': array([1.02142596, 1.05908132, 1.0289793 , 1.05190635, 1.03247023]),
 'score_time': array([0.04757476, 0.04686022, 0.06243205, 0.07127619, 0.04682779]),
 'test_score': array([0.68369124, 0.68772803, 0.67683888, 0.68081931, 0.68493946])}

In [60]:
# Metric required by the competition
f1_score(y_test,prediction,average='micro')

0.699502487562189

### Learn on  full training data and predict Test data

In [61]:
# Define X and y variables
X=pd.get_dummies(data.loc[:,:'has_secondary_use_other'])
y=data['damage'].astype(int)

In [62]:
clf=RandomForestClassifier()
clf.fit(X,y)


C:\Users\Preeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
# Read Test dataset
testdata=pd.read_csv('test_values.csv')
testdata.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [64]:
# Predict Damage_Grade for test dataset
prediction=clf.predict(pd.get_dummies(testdata))
print(np.unique(prediction))


[1 2 3]


In [65]:
pred=pd.DataFrame(prediction)
pred[0].value_counts()

2    57628
3    22302
1     6938
Name: 0, dtype: int64

In [66]:
# Format the prediction as per submission requirement
pred['building_id']=testdata['building_id']

pred.rename(columns={0:'damage_grade'},inplace=True)

pred=pred[['building_id','damage_grade']]
pred.head()


,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [68]:
# Save result to .csv file 
pred.to_csv('result.csv',index=False)